In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.tokenize import word_tokenize,sent_tokenize
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import gensim
from gensim import models

Using TensorFlow backend.


In [2]:
raw = pd.read_csv("data.csv",delimiter=",")

In [3]:
raw.isnull().sum()

category            1954
description         1459
gender              1088
sub_category           0
sub_sub_category       0
title               2068
dtype: int64

In [4]:
temp=raw.astype(str)

In [5]:
temp=temp.drop(temp[(temp.category=="nan")|(temp.category=="None")|(temp.category=="NaN")].index)
temp=temp.drop(temp[(temp.description=="nan")|(temp.description=="None")|(temp.description=="NaN")].index)
temp=temp.drop(temp[(temp.gender=="nan")|(temp.gender=="None")|(temp.gender=="NaN")].index)
temp=temp.drop(temp[(temp.sub_category=="nan")|(temp.sub_category=="None")|(temp.sub_category=="NaN")].index)
temp=temp.drop(temp[(temp.sub_sub_category=="nan")|(temp.sub_sub_category=="None")|(temp.sub_sub_category=="NaN")].index)
temp=temp.drop(temp[(temp.title=="nan")|(temp.title=="None")|(temp.title=="NaN")].index)

In [6]:
temp = temp[~temp.astype(str).eq('None').any(1)]

In [7]:
len(temp.category)

3398

In [8]:
temp.reset_index(drop=True)

,category,description,gender,sub_category,sub_sub_category,title
0,sports & outdoors,"Every time you step on to the field, you turn ...",unisex,team shop,team clothing,Miami Hurricanes adidas 2016 Football Coaches ...
1,sports & outdoors,"How to stencil a T-shirt? Print the stencil, c...",unisex,team shop,team clothing,New Era Philadelphia Phillies Red Stencil Box Tee
2,sports & outdoors,Proudly put your allegiance to the Kansas City...,unisex,team shop,team clothing,Men's Kansas City Royals Stitches Black/Camo P...
3,sports & outdoors,Sure you can go around and tell everyone how d...,unisex,team shop,team clothing,Chicago Blackhawks Women's Pond Hockey Pullove...
4,sports & outdoors,Is your flawless fashion sense something that ...,unisex,team shop,team clothing,Stadium Athletic Yale Bulldogs Navy Arch & Log...
5,clothing,"""Orlebar Brown """"Donald"""" shirt in waffle knit...",mens,tops,t-shirts,"Orlebar Brown Donald Waffle Cotton Polo Shirt,..."
6,clothing,Super-soft UA Tech fabric delivers incredible ...,womens,activewear,active tops,Women's UA Twist Tech V-Neck
7,kids & baby,"Smooth, stretch-woven fabric is ultra-lightwei...",boys,boys clothing,boys pants & shorts,Boys' UA Challenger Woven Shorts
8,sports & outdoors,Get the perfect look to let everyone know you ...,unisex,team shop,team clothing,Fanatics Branded Arizona State Sun Devils Kell...
9,sports & outdoors,Sure you can go around and tell everyone how d...,unisex,team shop,team clothing,Charlotte Hornets Fanatics Branded Women's Per...


In [9]:
category         = list(temp['category'])
description      = list(temp['description'])
sub_catedgory    = list(temp['sub_category'])
sub_sub_category = list(temp['sub_sub_category'])
title            = list(temp['title'])

In [10]:
def clean(holdtext):
    dataset=[]
    for i in tqdm(holdtext):        
        sent_tokenize_list = sent_tokenize(i)
        tokenized_reports = [word_tokenize(report) for report in sent_tokenize_list]
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        tokenized_reports_no_punctuation = []
        
        for review in tokenized_reports:
            new_review = []
            for token in review: 
                new_token = regex.sub(u'', token)
                if not new_token == u'':
                    new_review.append(new_token)
            tokenized_reports_no_punctuation.append(new_review)    
        tokenized_reports_no_stopwords = []
        for report in tokenized_reports_no_punctuation:
            new_term_vector = []
            for word in report:
                if not word in stopwords.words('english'):
                    new_term_vector.append(word)
            tokenized_reports_no_stopwords.append(new_term_vector)
        v=[]
        for i in tokenized_reports_no_stopwords:
            v.append(" ".join(i))
        gt=" ".join(v)
        text2=''.join([i for i in gt if not i.isdigit()])
        toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
        fnlTxt=''.join(toker.tokenize(text2))
        dataset.append(" ".join(fnlTxt.split()))
    return dataset

In [11]:
category,description,sub_catedgory,sub_sub_category,title = clean(category),clean(description),clean(sub_catedgory),clean(sub_sub_category),clean(title)
 

100%|██████████| 3398/3398 [00:08<00:00, 415.22it/s]


In [12]:
temp['gender'].unique()

array(['unisex', 'mens', 'womens', 'boys', 'baby & toddler', 'kids',
       'girls', 'plus size'], dtype=object)

In [13]:
mapping = {'unisex':0, 'mens':1, 'womens':2, 'boys':3, 'baby & toddler':4, 'kids':5,'girls':6, 'plus size':7}
gender=temp['gender'].map(mapping)

In [14]:
c1 = pd.DataFrame(category,columns=['category'])
c2 = pd.DataFrame(description,columns=['description'])
c3 = pd.DataFrame(sub_catedgory,columns=['sub_catedgory'])
c4 = pd.DataFrame(sub_sub_category,columns=['sub_sub_category'])
c5 = pd.DataFrame(title,columns=['title'])
c6 = pd.DataFrame(gender,columns=['gender'])
c1.reset_index(drop=True,inplace=True)
c2.reset_index(drop=True,inplace=True)
c3.reset_index(drop=True,inplace=True)
c4.reset_index(drop=True,inplace=True)
c5.reset_index(drop=True,inplace=True)
c6.reset_index(drop=True,inplace=True)

In [15]:
df = pd.concat([c1,c2,c3,c4,c5,c6],axis=1)

In [16]:
df.head(10)

,category,description,sub_catedgory,sub_sub_category,title,gender
0,sports outdoors,Every time step field turn lean mean training ...,team shop,team clothing,Miami Hurricanes adidas Football Coaches clima...,0
1,sports outdoors,How stencil Tshirt Print stencil cut stick shi...,team shop,team clothing,New Era Philadelphia Phillies Red Stencil Box Tee,0
2,sports outdoors,Proudly put allegiance Kansas City Royals disp...,team shop,team clothing,Men Kansas City Royals Stitches BlackCamo Pull...,0
3,sports outdoors,Sure go around tell everyone dedicated Chicago...,team shop,team clothing,Chicago Blackhawks Women Pond Hockey Pullover ...,0
4,sports outdoors,Is flawless fashion sense something maintain w...,team shop,team clothing,Stadium Athletic Yale Bulldogs Navy Arch Logo ...,0
5,clothing,Orlebar Brown Donald shirt waffle knit contras...,tops,tshirts,Orlebar Brown Donald Waffle Cotton Polo Shirt ...,1
6,clothing,Supersoft UA Tech fabric delivers incredible a...,activewear,active tops,Women UA Twist Tech VNeck,2
7,kids baby,Smooth stretchwoven fabric ultralightweight le...,boys clothing,boys pants shorts,Boys UA Challenger Woven Shorts,3
8,sports outdoors,Get perfect look let everyone know stylish Ari...,team shop,team clothing,Fanatics Branded Arizona State Sun Devils Kell...,0
9,sports outdoors,Sure go around tell everyone dedicated Charlot...,team shop,team clothing,Charlotte Hornets Fanatics Branded Women Perso...,0


In [79]:
Y = np.array(df['gender'].astype('int'))

In [80]:
X = df[['category','description','sub_catedgory','sub_sub_category','title']]

In [81]:
X = np.array(X)

In [82]:
''.join(X[0])

'sports outdoorsEvery time step field turn lean mean training machine Work style light Miami Hurricanes Football Coaches quarterzip jacket adidas The vibrant embroidered graphics make feel like training team whenever field Its climalite technology keep cool comfortable repping Miami Hurricanes Look feel like champ oneofakind Miami Hurricanes jacketteam shopteam clothingMiami Hurricanes adidas Football Coaches climalite QuarterZip Jacket Green'

In [83]:
holdText=[]
for i in range(len(X)):
    holdText.append(''.join(X[i]))

In [84]:
holdArray = np.array(holdText)

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

In [86]:
vect = CountVectorizer(max_features=3000)
vect.fit(holdArray)
simple_train_dtm = vect.transform(holdArray)
std   =  simple_train_dtm.toarray()

In [87]:
from sklearn.model_selection import train_test_split

In [88]:
X_train, X_test, y_train, y_test = train_test_split(std,Y, test_size=0.33, random_state=4)

In [89]:
import xgboost as xgb

In [90]:
model_xgboost = xgb.XGBClassifier() # boosted tree classifire

In [91]:
model_xgboost.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [92]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_recall_curve

In [93]:
pred = model_xgboost.predict(X_test)
print(accuracy_score(pred,y_test))

0.938502673797


In [94]:
confusion_matrix(y_test,pred)

array([[847,   2,  16,   0,   0,   0,   1,   0],
       [  2,  51,   9,   0,   0,   0,   0,   0],
       [ 12,  10, 129,   0,   0,   0,   0,   0],
       [  3,   0,   0,   5,   0,   0,   0,   0],
       [  0,   0,   0,   5,  11,   0,   2,   0],
       [  1,   0,   0,   0,   4,   0,   0,   0],
       [  0,   0,   1,   0,   0,   0,   6,   0],
       [  0,   0,   1,   0,   0,   0,   0,   4]])